In [ ]:
import pandas as pd
import numpy as np
import os,copy,itertools,ast
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdchem import Mol

In [ ]:
### Where reaxys file is stored
# ==================
reaxys_csv = 'datafiles/nazarov_reaxys.csv'
# ==================

for d in ['datafiles','tempfiles','outputfiles']:
    if not os.path.isdir(d):
        os.mkdir(d)

In [ ]:
def clean_smiles(smiles:str):
    """Return santiized SMILES"""
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def clean_rxn_smiles(rxn_smiles:str):
    """Return santiized SMILES for a reaction"""
    reactants,products = rxn_smiles.split('>>')
    if Chem.MolFromSmiles(reactants) is None: ### Skip over examples that RDKit cannot recognize
        return np.nan
    return f"""{'.'.join([clean_smiles(x) for x in reactants.split('.')])}>>{'.'.join([clean_smiles(x) for x in products.split('.')])}"""


# 1) Data loading, blank rxn removal, cleaning of rxn column, and prep of yield column

In [ ]:
### Read CSV and drop NAs
df_reaxys = pd.read_csv(reaxys_csv)
print(df_reaxys.shape,'Shape of CSV')
df_reaxys = df_reaxys.dropna(subset=['Reaction'])
print(df_reaxys.shape,'Shape after nan removal for Reaction column')

### Sanitize (clean) SMILES and remove any with invalid SMILES. Convert yields to floats. 
df_reaxys['rxn_clean_ReaxysOrder'] = df_reaxys.apply (lambda row: clean_rxn_smiles(row['Reaction']), axis=1)
df_reaxys['yield_list'] = df_reaxys.apply (lambda row: [float(x) for x in str(row['Yield (numerical)']).split(';')], axis=1)
df_reaxys = df_reaxys.dropna(subset=['rxn_clean_ReaxysOrder'])
print(df_reaxys.shape,'Shape after nan removal for rxn_clean_ReaxysOrder column')

df_reaxys

4812 rows × 43 columns
# 2) Identify Major Product and Reactants
## 2.1) X Products or Yield

In [ ]:
# ==================
num_prods = 1
# ==================

def get_mp_numORyield(rxn_smiles:str,yields:list,num_prods:int=1):
    """Return list of major product/s. Sort products by their yield, if available."""
    reactants,products = rxn_smiles.split('>>')
    reactants = reactants.split('.')
    products = products.split('.')
    products = [x for x in products if x not in reactants] ### Remove reagents that do not change through rxn
    prod_set = set()
    products = [x for x in products if not(x in prod_set or prod_set.add(x))] ### Remove any repeated products
    ### Return product lists if yield available
    if len(products) == num_prods:
        ### Return the list of products if it matches the num_prods count
        return [clean_smiles(products[x]) for x in range(0,num_prods)] 
    if len(yields) == len(products):
        ### Sort products in order of decreasing yield, keep num_prods number of products in list
        return [clean_smiles(x[1]) for x in sorted(zip(yields,products),reverse=True)[0:num_prods]]
    else:
        ### Return a list of products if there is no yield information to use
        return products

df_reaxys_mp = copy.copy(df_reaxys)
df_reaxys_mp['major_product_clean'] = df_reaxys_mp.apply (lambda row: get_mp_numORyield(row['rxn_clean_ReaxysOrder'],row['yield_list'],1), axis=1)
print(f"""Reactions with one product {len([x for x in df_reaxys_mp['major_product_clean'].to_list() if len(x) != 1])}""")
df_reaxys_mp.to_csv('tempfiles/df_reaxys_mp.csv',index=False)

In [ ]:
### This cell can be skipped
### This checks how many reactions are missing the major product, assuming that num_prods=1 in the above cell
df_id_check = copy.copy(df_reaxys_mp)
my_cols = list(df_id_check.columns.values)
df_id_check['major_product_clean_len'] = df_id_check.apply (lambda row: len(row['major_product_clean']), axis=1)
df_id_check = df_id_check[df_id_check.major_product_clean_len != 1]
df_id_check = df_id_check.drop_duplicates(subset='Reaction ID')
print(f'Missing major product for {df_id_check.shape[0]} reactions')

## 2.2) Get possible RXNs, ex: atom economy

In [ ]:
def get_rxns_AE(rxn_smiles:str,product_list:list,num_prods:int=1):
    """Return list of reaction SMILES that satisfy atom economy (<0.1 AMU difference between
    reactant and product mass)"""
    reactants,products = rxn_smiles.split('>>')
    reactants = reactants.split('.')
    
    if len(product_list) == 1: ### If major product was already found, create a reaction for it
        ### If 1 major product, create reaction SMILES for each reactant. 
        ### Only keep reactions if the difference in mass between reactant and product is less than 0.1 AMU
        return [f'{x}>>{product_list[0]}' for x in reactants if abs(Descriptors.MolWt(Chem.MolFromSmiles(x))-Descriptors.MolWt(Chem.MolFromSmiles(product_list[0])))<0.1]
    else:
        ### If multiple products, form reaction SMILES for each reactant and product combination
        ### Only return those whose reactant product mass difference is less than 0.1 AMU
        prod_react_combos = [[f'{x[0]}>>{x[1]}',abs(Descriptors.MolWt(Chem.MolFromSmiles(x[0]))-Descriptors.MolWt(Chem.MolFromSmiles(x[1])))] for x in itertools.product(reactants,product_list)]
        return [x[0] for x in prod_react_combos if x[1]<0.1]
    

df_reaxys_mp_AE = copy.copy(df_reaxys_mp)
df_reaxys_mp_AE['possible_rxns'] = df_reaxys_mp_AE.apply (lambda row: get_rxns_AE(row.['rxn_clean_ReaxysOrder'],row['major_product_clean']), axis=1)
print(df_reaxys_mp_AE.shape)
df_reaxys_mp_AE.to_csv('tempfiles/df_reaxys_mp_AE.csv',index=False)

## 2.3) Verify reactions

In [ ]:
def verify_nazarov(possible_rxns:list):
    """Verify Nazarov reactions by first checking for bonding patterns and then
    checking that the change in bonds between reactants and products is reasonable."""
    filtered_rxns = []
    for rxn in possible_rxns:
        reactants,products = rxn.split('>>')
        rmol = Chem.MolFromSmiles(reactants)
        if rmol is None:
            continue
        pmol = Chem.MolFromSmiles(products)
        
        ### Check 1: Detect bonding pattern and verify atoms
        sub_struc_matches = []
        ### Patterns allow for aromatic and double bonds
        patterns = ['[C,c]:[C,c]-[C,c]-[C,c]:[C,c]','[C,c]=[C,c]-[C,c]-[C,c]=[C,c]','[C,c]=[C,c]-[C,c]-[C,c]:[C,c]','[C,c]=[C,c]-[C,c]-[C,c]:[C,c]']
        for x in patterns:
            sub_struc_matches += list(rmol.GetSubstructMatches(Chem.MolFromSmarts(x)))
        final_rc = [] 
        for rc in sub_struc_matches:
            ### Skip repeat reaction centres or reaction centres in reverse order: 1,2==2,1
            if sorted(rc) and rc not in final_rc:
                ## If the middle atom is not SP2, cannot undergo Nazarov.
                ## Since pattern checks for carbon atoms, ensure that the middle atom is double bonded to a heavy atom (somewhat redundant)
                if str(rmol.GetAtomWithIdx(rc[2]).GetHybridization()) == 'SP2' and 1 not in [x.GetAtomicNum() for x in rmol.GetAtomWithIdx(rc[2]).GetNeighbors()]: 
                    final_rc.append(rc)         
        if len(final_rc) == 0:
            continue ### No reaction centres detected
                      
        ### Check 2: Detect bond change: -1 double bond, +1 single bond
        if len([x for x in rmol.GetBonds() if str(x.GetBondType()) == 'DOUBLE' or str(x.GetBondType()) == 'AROMATIC']) - len([x for x in pmol.GetBonds() if str(x.GetBondType()) == 'DOUBLE' or str(x.GetBondType()) == 'AROMATIC']) != 1: 
            continue
        if len([x for x in pmol.GetBonds() if str(x.GetBondType()) == 'SINGLE']) - len([x for x in rmol.GetBonds() if str(x.GetBondType()) == 'SINGLE']) != 2: 
            ### If the product does not have 2 more single bonds (bond forming and double bond that dissapears), reaction is invalid
            continue

        ### Check 3: RXNMapper Atom Mapping (Not shown as calculations and separate workflow required)
        filtered_rxns.append(rxn)
    return filtered_rxns
        

df_reaxys_mp_AE_rxn = copy.copy(df_reaxys_mp_AE)
df_reaxys_mp_AE_rxn['possible_rxns_verified'] = df_reaxys_mp_AE_rxn.apply (lambda row: verify_nazarov(row['possible_rxns']), axis=1)
print(f"""{len([x for x in df_reaxys_mp_AE_rxn['possible_rxns_verified'].to_list() if len(x) > 1])} reactions have multiple undecided outcomes""")
df_reaxys_mp_AE_rxn['possible_rxns_verified']

## 2.4) Remove Diasteroselectivity To Decide Undecided Reactions
Some reactions have had multiple possible reactions detected. To filter these, it can make sense to remove Diasteroselectivity. If stereochemistry is a must for EVERY reaction, then skip this part.

In [ ]:
def remove_stereo(possible_rxns:str):
    """If multiple products, return a unique list of reaction SMILES with chirality tags removed"""
    if len(possible_rxns) <= 1:
        return possible_rxns
    possible_rxns_noStereo = []
    for rxn in possible_rxns:
        reactants,products = rxn.split('>>')
        possible_rxns_noStereo.append(f'{Chem.MolToSmiles(Chem.MolFromSmiles(reactants),isomericSmiles=False)}>>{Chem.MolToSmiles(Chem.MolFromSmiles(products),isomericSmiles=False)}')
    return list(set(possible_rxns_noStereo))

df_reaxys_mp_AE_rxn_noSte = copy.copy(df_reaxys_mp_AE_rxn)
df_reaxys_mp_AE_rxn_noSte['possible_rxns_verified'] = df_reaxys_mp_AE_rxn_noSte.apply (lambda row: remove_stereo(row['possible_rxns_verified']), axis=1)
print(f"""{len([x for x in df_reaxys_mp_AE_rxn_noSte['possible_rxns_verified'].to_list() if len(x) > 1])} reactions have multiple undecided outcomes even after removing stereo""")
df_reaxys_mp_AE_rxn_noSte.to_csv('tempfiles/df_reaxys_mp_AE_rxn_noSte.csv',index=False)
df_reaxys_mp_AE_rxn_noSte['possible_rxns_verified']

# 3) Return conditions leading to highest yield and eliminate RXIDs where the reactants or products of the possible reactions differ beyond stereochemistry alone (aka: check that repeat RXIDs do not have different rxns)

In [ ]:
def get_clean_rxn_list(rxn_list:list):
    """Return a list of reaction SMILES that have been sanitized"""
    if len(rxn_list) > 1 or len(rxn_list) == 0:
        return ''
    reactants,products = rxn_list[0].split('>>')
    return f'{clean_smiles(reactants)}>>{clean_smiles(products)}'

def get_best_conditions(possible_rxns:list,yields:list):
    """Return index of the best reaction conditions"""
    if len(set([y for x in possible_rxns for y in x])) >1: ## If there are multiple different reactions for a RXID, check these manually
        return False
    if set([y for x in possible_rxns for y in x])==set(()):
        return 'Invalid'
    max_yields = [max(x) for x in yields]
    return max_yields.index(max(max_yields)) 

def get_sole_rxn(possible_rxns:list):
    """Check a list of reaction SMILES and ensure that they represent the same reaction
    even if some do not contain stereochemistry. Return the common reaction with chirality
    if possible."""
    rxns = list(set([y for x in possible_rxns for y in x]))
    if len(rxns) ==1: ## If there are multiple different reactions for a RXID, check these manually
        return rxns[0]
    elif len(set([x.split('>>')[0].replace('@','') for x in rxns])) != 1: ### check if non-stereo reactants are identical
        return False
    elif len(set([x.split('>>')[1].replace('@','') for x in rxns])) != 1: ## check if non-stereo products are identical
        return False ## if products differ in anything other than regio chemistry, return false
    elif len(set([x for x in rxns if '@' in x])) ==1:
        return list(set([x for x in rxns if '@' in x]))[0]
    else: ## 2 different stereochemistries are reported
        return 'Differing_Stereochemistry'

def get_condition(condition:str,index):
    """Return the best reaction conditions according to the index identified in get_best_conditions()"""
    if type(index) is int:
        return [max(x) for x in condition][index]
    else:
        return ''

df_reaxys_mp_AE_rxn_noSte_noRe = copy.copy(df_reaxys_mp_AE_rxn_noSte)

### Take reaction details for reaction with highest yield per rxid
rxn_detail_dic_ls = []
for col in list(df_reaxys_mp_AE_rxn_noSte_noRe.columns.values):
    rxn_detail_dic_ls.append(df_reaxys_mp_AE_rxn_noSte_noRe.groupby('Reaction ID')[col].apply(list).to_dict())
df_rxn_details_red = pd.DataFrame(rxn_detail_dic_ls)

df_reaxys_mp_AE_rxn_noSte['possible_rxns_verified'] = df_reaxys_mp_AE_rxn_noSte.apply (lambda row: remove_stereo(row['possible_rxns_verified']), axis=1)

### Combine each dataentry by its RXID
rxn_details_col_dic = {x:list(df_reaxys_mp_AE_rxn_noSte_noRe.columns.values)[x] for x in range(0,46)}
df_rxn_details_red_trans = df_rxn_details_red.transpose()
df_rxn_details_red_trans = df_rxn_details_red_trans.rename(columns=rxn_details_col_dic)
df_rxn_details_red_trans = df_rxn_details_red_trans.reset_index(drop=True)

### Keep reactions with best conditions and yields
df_rxn_details_red_trans['best_conditions'] = df_rxn_details_red_trans.apply (lambda row: get_best_conditions(row['possible_rxns_verified'],row['yield_list']), axis=1)
df_rxn_details_red_trans['best_yield'] = df_rxn_details_red_trans.apply (lambda row: get_condition(row['yield_list'],row['best_conditions']), axis=1)
df_rxn_details_red_trans['sole_rxn_ver_clean'] = df_rxn_details_red_trans.apply (lambda row: get_sole_rxn(row['possible_rxns_verified']), axis=1)
df_rxn_details_red_trans.to_csv('tempfiles/df_rxn_details_red_trans.csv')
verified_rxns = [x for x in df_rxn_details_red_trans['sole_rxn_ver_clean'].to_list() if x is not False and x is not 'Differing_Stereochemistry']
print(f'There are {len(verified_rxns)} reactions, {len(set(verified_rxns))} of which are unique')
df_rxn_details_red_trans

# 4) Remove RXIDs with repeat products
Some reactions that have been identified will turn out to be non-unique and need to be labelled such. The ID with the highest yield will be regarded as the True value

In [ ]:
def temp_change_noyields(yields):
    """If no yield is reported, return -10 so that it can be sorted out later"""
    if yields == '':
        return -10
    else:
        return yields

df_rxn_details_red_trans_unique = copy.copy(df_rxn_details_red_trans)

## Sort the dataframe by yield, this means that duplicates with lower yields can be dropped easily
df_rxn_details_red_trans_unique['best_yield_SORTONLY'] = df_rxn_details_red_trans_unique.apply (lambda row: temp_change_noyields(row['best_yield']), axis=1)
df_rxn_details_red_trans_unique = df_rxn_details_red_trans_unique.sort_values(by='best_yield_SORTONLY',ascending=False)

print(df_rxn_details_red_trans_unique.shape)
df_rxn_details_red_trans_unique = df_rxn_details_red_trans_unique.drop_duplicates(subset=['sole_rxn_ver_clean'])
print(df_rxn_details_red_trans_unique.shape)
df_rxn_details_red_trans_unique


# 5) Drop unnessecary columns and send to CSV

In [ ]:
def checkcomponent(rxn:str,component:int,banned_words=['Differing_Stereochemistry']):
    """Return component of reaction SMILES asumming nothing was flagged earlier"""
    if type(rxn) != str or rxn in banned_words:
        return False
    if Chem.MolFromSmiles(rxn.split('>>')[component]) is None:
        return False
    return rxn.split('>>')[component]

df_final = copy.copy(df_rxn_details_red_trans_unique)
df_final = df_final.drop(['best_yield_SORTONLY','possible_rxns_verified','possible_rxns','major_product_clean','yield_list','rxn_clean_ReaxysOrder'],axis=1)

### Add reactant and product column for ease of use later
df_final['Reactant_clean'] = df_final.apply (lambda row: checkcomponent(row['sole_rxn_ver_clean'],0), axis=1)
df_final['Product_clean'] = df_final.apply (lambda row: checkcomponent(row['sole_rxn_ver_clean'],1), axis=1)
df_final['rxnID'] = df_final.apply (lambda row: row['Reaction ID'][0], axis=1)
## export to csv
df_final.to_csv('tempfiles/nazarov_uniqueRXNs.csv',index=False)


# 6) Perform Atom-Mapping
This should be done using an atom-mapping software such as RXNMapper. The resulting CSV should contain a column with atom-mapped reaction SMILES. This column should be named 'rxnsmilesMap'.


# 7) Check Bond Changes

In [ ]:
def rm_map_and_clean_smiles(smiles:str):
    """Removing any atom mapping from a SMILES string"""
    my_mol = Chem.MolFromSmiles(smiles)
    atomMap = "molAtomMapNumber"
    for atom in my_mol.GetAtoms():
        if atom.HasProp(atomMap):
                atom.ClearProp(atomMap)
    return Chem.MolToSmiles(Chem.MolFromSmiles(Chem.MolToSmiles(my_mol)))


def detect_bond_changes(rmol:Mol,pmol:Mol):
    """Return a dictionary containing the atoms changing in a reaction.
    Atoms need to be mapped."""
    atom_changes = {}
    for n,atom in enumerate(rmol.GetAtoms()):
        ratom = {f'{sorted([x.GetBeginAtom().GetAtomMapNum(),x.GetEndAtom().GetAtomMapNum()])}':str(x.GetBondType()) for x in atom.GetBonds()}
        patom = {f'{sorted([x.GetBeginAtom().GetAtomMapNum(),x.GetEndAtom().GetAtomMapNum()])}':str(x.GetBondType()) for x in pmol.GetAtomWithIdx(atom.GetAtomMapNum()).GetBonds()}
        ### if the bond types and atoms involved in bonds change for an atom, record this in a dictionary of atom changes
        if sorted(ratom.keys()) != sorted(patom.keys()):
            atom_changes[pmol.GetAtomWithIdx(atom.GetAtomMapNum()).GetAtomMapNum()] = patom
            continue
        if sorted(ratom.values()) != sorted(patom.values()):
            atom_changes[pmol.GetAtomWithIdx(atom.GetAtomMapNum()).GetAtomMapNum()] = patom
            continue
    return atom_changes

def identify_RC_overlap(rmol:Mol,pmol:Mol,atom_changes:dict,substructSMARTS:str='[C,c]=[C,c]-[C,c](-[C,c]=[C,c])=[O]'): 
    """Identify possible reaction centres and verify these by ensuring that the reaction centres
    contain at least 4 atoms that are undergoing changes during the reaction. 
    Returns a list of viable reaction centres."""
    possible_rcs = [[y for y in x] for x in rmol.GetSubstructMatches(Chem.MolFromSmarts(substructSMARTS))] 
    
    ### For each of the possible reaction centres, verify that the bonding is correct. 
    ### Note: this assumes a divinyl ketone bonding pattern
    valid_possible_rcs = [] 
    for rc in possible_rcs: 
        if rmol.GetBondBetweenAtoms(rc[0],rc[-2]) is None:
            rc_bond_list = [rmol.GetBondBetweenAtoms(rc[x],rc[x-5]) for x in range(0,4)] 
            rc_bond_list += [rmol.GetBondBetweenAtoms(rc[2],rc[-1])]
            if None in rc_bond_list:
                rc_bond_list.remove(None)
            if len(rc_bond_list) == 5:
                valid_possible_rcs.append(rc)
    valid_possible_rcs = [[rmol.GetAtomWithIdx(y).GetAtomMapNum() for y in x] for x in valid_possible_rcs]

    ### If atommapping has errors, this checks that the RCs found for the reactants does indeed match the products
    ### The atom mapping in the reactants matches the atom indexes in the product
    valid_possible_rcs_p = []
    for rc in valid_possible_rcs:
        if pmol.GetBondBetweenAtoms(rc[0],rc[-2]) is not None:
            rc_bond_list = [pmol.GetBondBetweenAtoms(rc[x],rc[x-5]) for x in range(0,4)] 
            rc_bond_list += [pmol.GetBondBetweenAtoms(rc[2],rc[-1])]
            if None in rc_bond_list:
                rc_bond_list.remove(None)
            if len(rc_bond_list) == 5:
                valid_possible_rcs_p.append(rc)
    ### Convert reaction centre atom list to a string and dictionary key. The value becomes the number of atoms in the
    ### reaction centre that are undergoing a change according to atom mapping analysis.
    overlap_rc = {','.join([str(y) for y in x]):len(set(x)-set(list(atom_changes.keys()))) for x in valid_possible_rcs_p}
    ### Return reaction centres that have at least 4 atoms that are undergoing changes through the reaction
    return [[int(x) for x in k.split(',')] for k,v in overlap_rc.items() if v == min(overlap_rc.values()) and v <=4]


def get_likely_RCs(rxn:str,rxn_templates:list=['[C,c]=[C,c]-[C,c](-[C,c]=[C,c])=[O]','[C,c]:[C,c]-[C,c](-[C,c]=[C,c])=[O]','[C,c]=[C,c]-[C,c](-[C,c]:[C,c])=[O]','[C,c]:[C,c]-[C,c](-[C,c]:[C,c])=[O]']):
    """Taking a list of possible reaction centre bonding patterns, identify and verify these reaction centres.
    Return a list containing these different reaction centres or a string if there are none"""
    rsmiles,psmiles = rxn.split('>>')
    rmol = Chem.MolFromSmiles(rsmiles)
    pmol = Chem.MolFromSmiles(psmiles)
    atom_changes = detect_bond_changes(rmol,pmol)
    ### For each bonding pattern, identify viable and verified reaction ceentres
    reaction_centres = []
    for template in rxn_templates:
        reaction_centres += identify_RC_overlap(rmol,pmol,atom_changes,substructSMARTS=template)
    final_rc = {} ### remove repeats that are ordered in reverse: 1,2,3 == 3,2,1
    for rc in reaction_centres:
        if sorted(rc) not in list(final_rc.keys()):
            final_rc[','.join([str(x) for x in sorted(rc)])] = rc
    if len(final_rc)>0:
        return list(final_rc.values())
    else:
        return 'No options'

def get_atomchanges(rxn:str):
    """Return a list of atoms that undergo changes in a reaction"""
    rsmiles,psmiles = rxn.split('>>')
    rmol = Chem.MolFromSmiles(rsmiles)
    pmol = Chem.MolFromSmiles(psmiles)
    return list(detect_bond_changes(rmol,pmol).keys())


In [ ]:
df_rc = pd.read_csv('tempfiles/nazarov_RCs.csv')
nazarov_templates = ['[C,c]=[C,c]-[C,c](-[C,c]=[C,c])=[O]','[C,c]:[C,c]-[C,c](-[C,c]=[C,c])=[O]','[C,c]=[C,c]-[C,c](-[C,c]:[C,c])=[O]','[C,c]:[C,c]-[C,c](-[C,c]:[C,c])=[O]']
df_rc['reaction_centre_MAPatoms'] = df_rc.apply (lambda row: get_likely_RCs(row['rxnsmilesMap'],nazarov_templates), axis=1)
df_rc['atomchanges'] = df_rc.apply (lambda row: get_atomchanges(row['rxnsmilesMap']), axis=1)

df_rc.to_csv('outputfiles/nazarov_RCs.csv',index=False)

# 8) Manual Identification
#### The following 86 IDs must have their reaction centres found manually:
[8643211, 9216076, 9216179, 9216342, 9216869, 9217005, 9217087, 9363890, 9487353, 9489938, 9618519, 11275489, 11275490, 11275497, 27934219, 29903574, 35300994, 35300995, 35300996, 35300997, 35300998, 35300999, 35301000, 35301001, 35791065, 37216716, 37216717, 37216718, 37216719, 37216720, 37226535, 37226538, 37226539, 37226540, 38104147, 41930398, 41930399, 41930400, 41930401, 41930402, 41930403, 41930405, 41930407, 41930408, 41930409, 41930411, 41930442, 41930443, 41930444, 41930445, 41930446, 41930447, 41930448, 41930451, 41930454, 42000161, 42000690, 43631701, 43631702, 45063873, 48760477, 48760480, 48760483, 48760485, 48760486, 48760487, 48760488, 48760489, 48760490, 48760493, 48760494, 48760495, 48760496, 48760497, 48760498, 49762243, 49762291, 51417000, 51417004, 51417008, 51417048, 51417050, 51417051, 56260136, 56887736, 60835617]